In [1]:
import functions as db
import pandas as pd
# Similarity
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
users_df = db.get_users_df()
users_df.head()

,id,name,email,username,password,phone,fav_author
0,1424,Mechelle,test1@gmail.com,username_1,12345678A,999999999,William Shatner
1,1848,Tyesha,test2@gmail.com,username_2,12345678A,999999999,Meggin Cabot
2,3363,Dean,test3@gmail.com,username_3,12345678A,999999999,Rebecca Wells
3,4017,Margareta,test5@gmail.com,username_5,12345678A,999999999,Sharon Creech
4,6345,Meghann,test6@gmail.com,username_6,12345678A,999999999,Caiseal Mor


In [3]:
books_df = db.get_books_df(None, None)
books_df.head()

,id,title,author,pub_year,publisher,price,avg_rate,count_rate,image
0,6379702,"My \Star Trek\"" Memories""",William Shatner,1993,HarperCollins Publishers,17.95,6.0,3,http://images.amazon.com/images/P/0006379702.0...
1,34544003,Southampton Row (Charlotte &amp; Thomas Pitt N...,Anne Perry,2002,Ballantine Books,34.00,0.0,0,http://images.amazon.com/images/P/034544003X.0...
2,34545104,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,20.00,0.0,0,http://images.amazon.com/images/P/034545104X.0...
3,38550120,A Painted House,JOHN GRISHAM,2001,Doubleday,18.00,7.0,1,http://images.amazon.com/images/P/038550120X.0...
4,52165615,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,33.00,3.0,1,http://images.amazon.com/images/P/052165615X.0...


In [5]:
ratings_df = db.get_ratings_df()
ratings_df.head(), len(ratings_df)

(   id  User_id   Book_id  rate
 0   1    11676   6379702   5.0
 1   2    63714   6379702   8.0
 2   3   276798   6379702   5.0
 3   4     3363  60096195   0.0
 4   5     7125  60096195   8.0,
 81)

In [6]:
ratings_df[ratings_df['User_id'] == 276798]

,id,User_id,Book_id,rate
2,3,276798,6379702,5.0


In [7]:
ratings_df[ratings_df['User_id'] == 11676]

,id,User_id,Book_id,rate
0,1,11676,6379702,5.0
21,22,11676,91830893,0.0
60,61,11676,671749609,7.0


In [8]:
# get most rated books
def get_most_rated_books(year=None, author=None, limit=10):
    return sorted(db.get_books(year, author), key=lambda b: -b.avg_rate)[:limit]

In [9]:
a = ['Meggin Cabot', 'Roddy Doyle']
year = None
results = []
for author in a:
    results.extend(get_most_rated_books(year, author))
results

[<Book(id=60096195, author=Meggin Cabot, title=The Boy Next Door, price=15.0, year=2002, avg_rate=7.0)>,
 <Book(id=3596151465, author=Roddy Doyle, title=Henry der Held., price=43.0, year=2001, avg_rate=10.0)>]

In [10]:
# Number of users
print('The ratings dataset has', ratings_df['User_id'].nunique(), 'unique users')

# Number of books
print('The ratings dataset has', ratings_df['Book_id'].nunique(), 'unique books')

# Number of ratings
print('The ratings dataset has', ratings_df['rate'].nunique(), 'unique ratings')

# List of unique ratings
print('The unique ratings are', sorted(ratings_df['rate'].unique()))

The ratings dataset has 60 unique users
The ratings dataset has 43 unique books
The ratings dataset has 8 unique ratings
The unique ratings are [0.0, 3.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]


In [11]:
# tested but didn't work
# https://colab.research.google.com/drive/1xxijmbGTqLGVGKruAcVHPODCIyxfw09-#scrollTo=JlxIlzJaMlgc

In [15]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import SVD, Dataset
from surprise import SVDpp
from surprise import KNNBasic
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV

In [21]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings_df[['User_id', 'Book_id', 'rate']], reader)

In [22]:
data

In [23]:
trainset = data.build_full_trainset()
trainset

In [34]:
param_grid = {'n_factors': [10, 20, 40, 100, 150],
              'n_epochs': [1, 5, 10, 20,25,30],
              'lr_all':[0.001, 0.005,0.01,0.1],
              'reg_all':[0.001, 0.02,0.05,0.1]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=2)
grid_search.fit(data)     

In [35]:
grid_search.best_score

{'rmse': 3.99661659856209, 'mae': 3.5977466849955033}

In [36]:
grid_search.best_params

{'rmse': {'n_factors': 150, 'n_epochs': 25, 'lr_all': 0.005, 'reg_all': 0.001},
 'mae': {'n_factors': 10, 'n_epochs': 25, 'lr_all': 0.1, 'reg_all': 0.02}}

In [55]:
algo = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.05)
trainset = data.build_full_trainset()
testset = trainset.build_testset()
# fitting algorithm
algo.fit(trainset)

In [56]:
preds = algo.test(testset)

In [57]:
accuracy.rmse(preds)

RMSE: 2.5573


2.557290366507372

In [58]:
trainSet, testSet = train_test_split(data, test_size=0.2)

In [59]:
testSet

[(276820, 140062718, 0.0),
 (276780, 8434811634, 0.0),
 (21364, 140062718, 0.0),
 (34491, 140279091, 5.0),
 (276751, 3596218098, 8.0),
 (276762, 380000059, 0.0),
 (276733, 2080674722, 0.0),
 (34509, 671749609, 8.0),
 (11676, 6379702, 5.0),
 (276747, 609801279, 0.0),
 (276746, 425115801, 0.0),
 (25454, 140062718, 8.0),
 (276772, 3596151465, 10.0),
 (3363, 64405176, 10.0),
 (276774, 99543818, 0.0),
 (276804, 64405176, 0.0),
 (31919, 140062718, 0.0)]

In [27]:
algo.test(testSet)

[Prediction(uid=9226, iid=345443683, r_ui=9.0, est=6.542223126257719, details={'was_impossible': False}),
 Prediction(uid=276726, iid=155061224, r_ui=5.0, est=4.654982523569042, details={'was_impossible': False}),
 Prediction(uid=276748, iid=747558167, r_ui=6.0, est=5.0086145961896555, details={'was_impossible': False}),
 Prediction(uid=276762, iid=380000059, r_ui=0.0, est=2.784643195454671, details={'was_impossible': False}),
 Prediction(uid=276796, iid=330332775, r_ui=5.0, est=4.653843504303772, details={'was_impossible': False}),
 Prediction(uid=18587, iid=60517794, r_ui=0.0, est=3.388376330892182, details={'was_impossible': False}),
 Prediction(uid=278100, iid=61054143, r_ui=0.0, est=2.5323949519436986, details={'was_impossible': False}),
 Prediction(uid=276729, iid='052165615X', r_ui=3.0, est=4.095140483244991, details={'was_impossible': False}),
 Prediction(uid=276746, iid='055356451X', r_ui=0.0, est=2.092728740609238, details={'was_impossible': False}),
 Prediction(uid=276733, i

In [36]:
# https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system#Collaborative-Filtering
# https://www.alldatascience.com/recommender-systems/simple-recipe-recommender-system-with-scikit-surprise/
def get_recommendations_SVD(user_id, max_recommendations=10, thresh=0.0):
    # setup reader
    reader = Reader(rating_scale=(1, 10))
    data = Dataset.load_from_df(ratings_df[['user_id', 'book_id', 'rate']], reader)
    # initate algorithm
    algo = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.05)
    # fitting algorithm
    algo.fit(data.build_full_trainset())
    # predict user rating on the books
    rated_books = ratings_df[ratings_df['user_id'] == user_id]['book_id'].tolist()
    predictions = []
    for book_id in books_df['id']:
        if(book_id in rated_books):  # predict only unrated books
            continue
        predictions.append(algo.predict(user_id, book_id))
    dfx = pd.DataFrame(predictions, columns=['uid', 'iid', 'r_ui', 'est', 'details'])
    dfx = dfx.sort_values(['est'], ascending=False)
    print(dfx)
    recommended_books_ids = dfx[dfx['est'] >= thresh][:max_recommendations]['iid'].tolist()
    return [db.get_book_by_id(book_id) for book_id in recommended_books_ids]

In [39]:
get_recommendations_SVD(7346, thresh=-1)

     uid         iid  r_ui       est                    details
10  7346   345443683  None  5.858714  {'was_impossible': False}
16  7346   446520802  None  5.195449  {'was_impossible': False}
38  7346  3499230933  None  5.141816  {'was_impossible': False}
15  7346   440414121  None  5.082130  {'was_impossible': False}
0   7346     6379702  None  5.069705  {'was_impossible': False}
40  7346  3596218098  None  5.019184  {'was_impossible': False}
39  7346  3596151465  None  4.986912  {'was_impossible': False}
28  7346   671749609  None  4.936797  {'was_impossible': False}
31  7346   747558167  None  4.929349  {'was_impossible': False}
12  7346   380711524  None  4.898044  {'was_impossible': False}
1   7346    60517794  None  4.870193  {'was_impossible': False}
27  7346   671537458  None  4.863578  {'was_impossible': False}
30  7346   684867621  None  4.858624  {'was_impossible': False}
29  7346   679776818  None  4.856609  {'was_impossible': False}
4   7346    91830893  None  4.840883  {'

[<Book(id=345443683, author=ANNE RICE, title=Blackwood Farm (Rice, Anne, Vampire Chronicles.), price=23.0, year=2003, avg_rate=8.66666666666667)>,
 <Book(id=446520802, author=Nicholas Sparks, title=The Notebook, price=17.0, year=1996, avg_rate=6.66666666666667)>,
 <Book(id=3499230933, author=Kathrine Kressmann Taylor, title=Adressat unbekannt., price=18.0, year=2002, avg_rate=10.0)>,
 <Book(id=440414121, author=CHRISTOPHER PAUL CURTIS, title=The Watsons Go to Birmingham - 1963 (Yearling Newbery), price=43.0, year=1997, avg_rate=10.0)>,
 <Book(id=6379702, author=William Shatner, title=My \Star Trek\" Memories", price=17.95, year=1993, avg_rate=6.0)>,
 <Book(id=3596218098, author=Carlos Castaneda, title=Reise nach Ixtlan. Die Lehre des Don Juan., price=22.95, year=1998, avg_rate=8.0)>,
 <Book(id=3596151465, author=Roddy Doyle, title=Henry der Held., price=43.0, year=2001, avg_rate=10.0)>,
 <Book(id=671749609, author=Patrick Suskind, title=PERFUME : PERFUME, price=25.0, year=1991, avg_rat

In [22]:
ratings_df[ratings_df['book_id'] == 345443683]

,id,rate,user_id,book_id
36,37,9.0,1848,345443683
37,38,9.0,9226,345443683
38,39,8.0,276788,345443683


In [23]:
ratings_df[ratings_df['user_id'] == 7346]

,id,rate,user_id,book_id
5,6,8.0,7346,60096195
